In [1]:
import os
import shutil
import sys
import pathlib
import pickle as pkl
import json
import argparse

import numpy as np
import pandas as pd

import torch
import torchvision

from torch.utils.data import DataLoader, Subset

from datasets import deep_fashion_ctsrbm
from arch import backbones, models, heads

from tqdm import tqdm

import utils.mem
import utils.list
import utils.train
import utils.time
import utils.log
import utils.dict
import utils.sig
import utils.pkl
import utils.chunk
import utils.arr

from time import time
from datetime import datetime

from itertools import chain
from functools import reduce

import json
import socket

import pprint

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

---

In [2]:
ctsrbm_dataset_dir = os.path.join(pathlib.Path.home(), "data", "DeepFashion", "Consumer-to-shop Clothes Retrieval Benchmark")
ctsrbm_dataset = deep_fashion_ctsrbm.ConsToShopClothRetrBmkImageLoader(ctsrbm_dataset_dir, img_transform=None)

In [3]:
def load_json_data(
        json_data_filename
        ):

    with open(json_data_filename, 'r') as json_data_file:
        json_data = json.load(json_data_file)

    return json_data

In [4]:
eval_data_filename_1 = os.path.join(
    pathlib.Path.home(), 
    "data",
    "fashion_retrieval",
    "ret_train",
    "resnet_50",
    "test_006",
    "eval_ctsrbm_accuracy__data.json"
)

eval_data_1 = load_json_data(eval_data_filename_1)

In [5]:
eval_data_filename_2 = os.path.join(
    pathlib.Path.home(), 
    "data",
    "fashion_retrieval",
    "ret_train",
    "convnext_t",
    "test_002",
    "eval_ctsrbm_accuracy__data.json"
)

eval_data_2 = load_json_data(eval_data_filename_2)

---

In [6]:
train_cons_idxs = ctsrbm_dataset.get_subset_indices(split="train", domain="consumer")
val_cons_idxs = ctsrbm_dataset.get_subset_indices(split="val", domain="consumer")
test_cons_idxs = ctsrbm_dataset.get_subset_indices(split="test", domain="consumer")

cons_idxs = np.concatenate([train_cons_idxs, val_cons_idxs, test_cons_idxs])
num_cons_imgs = len(cons_idxs)

In [7]:
acc_array_1 = np.empty(num_cons_imgs, dtype="float")
acc_array_2 = np.empty(num_cons_imgs, dtype="float")
num_array = np.empty(num_cons_imgs, dtype="int")

zidx = 0
for idx, acc, num  in eval_data_1["results"]["train"]:
    acc_array_1[zidx] = acc
    num_array[zidx] = num
    zidx += 1
for idx, acc, num  in eval_data_1["results"]["val"]:
    acc_array_1[zidx] = acc
    num_array[zidx] = num
    zidx += 1
for idx, acc, num  in eval_data_1["results"]["test"]:
    acc_array_1[zidx] = acc
    num_array[zidx] = num
    zidx += 1

zidx = 0
for idx, acc, num in eval_data_2["results"]["train"]:
    acc_array_2[zidx] = acc
    zidx += 1
for idx, acc, num  in eval_data_2["results"]["val"]:
    acc_array_2[zidx] = acc
    zidx += 1
for idx, acc, num  in eval_data_2["results"]["test"]:
    acc_array_2[zidx] = acc
    zidx += 1

In [8]:
split_arr =\
    ["train"] * len(train_cons_idxs) +\
    ["val"] * len(val_cons_idxs) +\
    ["test"] * len(test_cons_idxs)

results_df = pd.DataFrame(
    data = {
        "img_idxs": cons_idxs,
        "split": split_arr,
        "acc_1": acc_array_1,
        "acc_2": acc_array_2,
        "num_shop_imgs": num_array
    }
)

In [9]:
results_df = results_df.copy()

results_df["acc_diff"] = results_df["acc_2"] - results_df["acc_1"]
results_df = results_df.sort_values("acc_diff", ascending=False)

In [10]:
results_df.head(10)

,img_idxs,split,acc_1,acc_2,num_shop_imgs,acc_diff
52258,129155,train,0.0,1.0,1,1.0
102611,23748,val,0.0,1.0,1,1.0
121729,119573,val,0.0,1.0,1,1.0
73532,180248,train,0.0,1.0,1,1.0
121734,119578,val,0.0,1.0,1,1.0
73530,180246,train,0.0,1.0,1,1.0
121736,119581,val,0.0,1.0,1,1.0
121737,119582,val,0.0,1.0,1,1.0
27997,68818,train,0.0,1.0,1,1.0
121799,120055,val,0.0,1.0,1,1.0


In [11]:
results_df.tail(10)

,img_idxs,split,acc_1,acc_2,num_shop_imgs,acc_diff
17852,44761,train,1.0,0.0,1,-1.0
164262,87022,test,1.0,0.0,1,-1.0
38677,94079,train,1.0,0.0,2,-1.0
46020,112836,train,1.0,0.0,1,-1.0
170714,119312,test,1.0,0.0,1,-1.0
46017,112833,train,1.0,0.0,1,-1.0
102753,24491,val,1.0,0.0,1,-1.0
46016,112832,train,1.0,0.0,1,-1.0
54749,135106,train,1.0,0.0,1,-1.0
122905,124861,val,1.0,0.0,1,-1.0


---

In [12]:
num_cons_imgs = 20
split = "train"

###

better_idxs_1 = list(results_df[results_df["split"] == split].tail(num_cons_imgs)["img_idxs"])
better_idxs_2 = list(results_df[results_df["split"] == split].head(num_cons_imgs)["img_idxs"])

better_idxs = better_idxs_1 + better_idxs_2
better_idxs.sort()

###

print("\"desired_cons_img_idxs\": " + str(better_idxs) + ",\n")

"desired_cons_img_idxs": [15120, 15261, 15369, 15442, 15443, 15444, 15447, 15450, 15476, 44761, 58510, 68818, 69024, 94079, 112617, 112822, 112823, 112824, 112832, 112833, 112836, 129155, 135106, 171278, 180246, 180248, 180266, 180312, 180330, 180343, 180498, 207745, 227322, 227574, 227582, 227584, 227612, 227727, 227729, 227731],



In [13]:
num_cons_imgs = 20
split = "val"

###

better_idxs_1 = list(results_df[results_df["split"] == split].tail(num_cons_imgs)["img_idxs"])
better_idxs_2 = list(results_df[results_df["split"] == split].head(num_cons_imgs)["img_idxs"])

better_idxs = better_idxs_1 + better_idxs_2
better_idxs.sort()

###

print("\"desired_cons_img_idxs\": " + str(better_idxs) + ",\n")

"desired_cons_img_idxs": [23560, 23748, 23751, 24106, 24107, 24108, 24111, 24320, 24491, 60515, 60522, 109113, 109269, 109492, 119196, 119199, 119200, 119547, 119572, 119573, 119578, 119581, 119582, 120055, 120067, 120068, 120070, 120073, 124861, 136668, 136669, 136998, 137002, 137003, 161343, 176265, 207163, 207168, 207336, 207338],



In [14]:
num_cons_imgs = 20
split = "test"

###

better_idxs_1 = list(results_df[results_df["split"] == split].tail(num_cons_imgs)["img_idxs"])
better_idxs_2 = list(results_df[results_df["split"] == split].head(num_cons_imgs)["img_idxs"])

better_idxs = better_idxs_1 + better_idxs_2
better_idxs.sort()

###

print("\"desired_cons_img_idxs\": " + str(better_idxs) + ",\n")

"desired_cons_img_idxs": [8468, 44630, 60783, 87022, 95418, 95488, 95489, 95491, 95493, 95497, 95856, 96433, 96438, 96488, 96595, 119312, 119449, 119894, 119895, 119896, 119902, 147135, 168402, 168418, 168491, 168508, 168615, 168617, 168621, 168660, 210756, 210871, 210986, 211010, 211120, 211126, 211132, 211136, 211374, 211375],

